# VOLTTRON Bacnet Collector Notebook

This notebook sets up a Bacnet device (or gateway) and forwards data
from one VOLLTRON instance (this Collector) to another instance (the Aggregator).

Most of the notebook's setup and execution is done with shell commands, called from Python.

# Setup: Prepare the Volttron Environment

VOLTTRON must be installed before using this notebook. For detailed instructions on
installing and configuring a VOLTTRON/Jupyter server environment, see [Jupyter Notebooks](http://volttron.readthedocs.io/en/devguides/supporting/utilities/JupyterNotebooks.html) 
in VOLTTRON ReadTheDocs.

As is described in that guide, environment variables should have been defined before starting 
the Jupyter server:

````
$ export VOLTTRON_ROOT=~/repos/volttron
````
        (path of the VOLTTRON repository, installed prior to running bootstrap)

````
$ export VOLTTRON_HOME=~/.volttron
````
        (directory in which the VOLTTRON instance runs)

The first VOLTTRON instance on a server usually runs, by convention, in ~/.volttron.
If multiple VOLTTRON instances are to be run on a single host, each must have its own VOLTTRON_HOME.

Also before starting the Jupyter server, a VOLTTRON virtual environment should have been 
activated by executing the following in $VOLTTRON_ROOT:

````
$ source env/bin/activate
````

The Python code below does some initialization to prepare for the steps that follow.

In [ ]:
import datetime
import json
import os
import pprint
import sqlite3
import subprocess
import sys
import time

# Define a "run this shell command" method, wrapping subprocess.check_output()
def _sh(shell_command, shell=True, stderr=None):
    try:
        return_value = subprocess.check_output(shell_command, shell=shell, stderr=stderr)
    except Exception, err:
        print('Shell command failed: {}', shell_command)
        print(err)
        return_value = 'Error'
    return return_value

# Same as _sh(), except that this also prints the command output, preceded by an optional label.
def _print_sh(shell_command, label=None, **kwargs):
    print('{0}: {1}\n'.format(label+':' if label else '', _sh(shell_command, **kwargs)))

# Set up local variables vhome and vroot.
# The environment variables VOLTTRON_ROOT and VOLTTRON_HOME should already be defined -- see above.
vroot = %env VOLTTRON_ROOT
vhome = %env VOLTTRON_HOME
print("VOLTTRON_ROOT={}".format(vroot))
print("VOLTTRON_HOME={}".format(vhome))

# Define a VIP_SOCKET environment variable for use while installing and running agents.
socket_name = 'ipc://' + vhome + '/run/vip.socket'
%env VIP_SOCKET=$socket_name

# Run from the VOLTTRON root directory.
os.chdir(vroot)

print("Initialization complete")

# Setup: Shut Down All Agents

This ensures a clean agent installation process by the notebook.

In [ ]:
print('Wait for the list to be displayed, and confirm that no agents are listed as running...\n')

# Shut down all agents.
_sh('volttron-ctl shutdown')

# List agent status to verify that the status of each agent is 0 or blank.
_print_sh('volttron-ctl status', stderr=subprocess.STDOUT)

# Setup: Discover the Collector's Network Parameters

In order for this Collector to forward data to an Aggregator, the Aggregator
must know the Collector's network parameters, storing them in its known_hosts file.
Discover those parameters now.

Copy the vip-address's IP and port, and the serverkey,
to the Aggregator notebook under 
'Setup: Add Each Collector to the known_hosts File',
and execute that notebook's code to add this Collector to known_hosts.

In [ ]:
# Obtain this server's IP address, volttron port number (usually 22916), and server key:
print('Obtaining network parameters and server key; please wait...\n')
_print_sh('curl ifconfig.me', label='Public IP address')
_print_sh('volttron-ctl auth serverkey', label='Serverkey')
_print_sh('cat {}/config'.format(vhome), label='Config file')

# Setup: Configure the Aggregator's Network Parameters

This Collector forwards data to an Aggregator, so it must be
configured with the Aggregator's IP address, port number and server key.

Define those parameters here. 

Obtain them from the Aggregator notebook,
'Setup: Discover the Aggregator's Network Parameters'.

In [ ]:
aggregator_vip_address = '54.67.31.234'
aggregator_vip_port = '22916'
aggregator_server_key = 'A_WyNaTRQu3jkMeX6NgmchCCnPsYhZUjnt2zdAyf0HU'

aggregator_vip = "tcp://{0}:{1}".format(aggregator_vip_address, aggregator_vip_port)

print('vip = {0}'.format(aggregator_vip))
print('aggregator_server_key = {0}'.format(aggregator_server_key))

# Setup: Test the TCP Connection

The ForwardHistorian will send requests to the VOLTTRON Aggregator instance
via TCP commands. Test that the Aggregator instance is capable of receiving
TCP requests on the designated IP address and port.

If this test fails, the port may not be open on the other server (firewall issue?),
the request may be for the wrong IP address and/or port ID,
or the other server's VOLTTRON instance may be down or incorrectly configured.

In [ ]:
# Use an 'nc' (netcat) command to test the TCP connection
shell_command = 'nc -z -vv -w5 {0} {1}'.format(aggregator_vip_address, aggregator_vip_port)
_print_sh(shell_command, label='Network connection test result', stderr=subprocess.STDOUT)

# Setup: Configure a ForwardHistorian

Create a configuration file for this collector's ForwardHistorian.

The file specifies the Aggregator's IP address, port and server key,
and indicates which topics should be forwarded.

In [ ]:
config = """{{
    "destination-vip": "{0}",
    "destination-serverkey": "{1}",
    "required_target_agents": [],
    "custom_topic_list": [],
    "services_topic_list": ["devices"],
    "topic_replace_list": [
        {{
            "from": "FromString", 
            "to": "ToString"
        }}
    ]
}}""".format(aggregator_vip, aggregator_server_key)
print("config = {}".format(config))
config_path = vhome + '/my_bacnet_forwarder.config'
with open(config_path, 'w') as file:
    file.write(config)
print('Forwarder configuration written to {}'.format(config_path))

# Setup: Run a BACnet Scan to Discover BACnet Devices

TBD: Explain what this is and how to configure it...

In [ ]:
volttron_ip = '192.168.14.65/24'

bacpypes_ini_content = """[BACpypes]
objectName: Betelgeuse
address: {0}
objectIdentifier: 599
maxApduLengthAccepted: 1024
segmentationSupported: segmentedBoth
vendorIdentifier: 15""".format(volttron_ip)
print('bacpypes_ini_content = {}'.format(bacpypes_ini_content))

bacpypes_ini_path = vroot + '/scripts/bacnet/BACpypes.ini'
with open(bacpypes_ini_path, 'w') as file:
    file.write(bacpypes_ini_content)
print('BACpypes ini file written to {}\n'.format(bacpypes_ini_path))

print('Running bacnet_scan, wait for output...')
os.chdir(vroot + '/scripts/bacnet/')
_print_sh('python {0}'.format(vroot + '/scripts/bacnet/bacnet_scan.py'))
os.chdir(vroot)

# Configure a BACnet Proxy Agent

TBD: Explain what the proxy is and how it works...

Other than defining the virtual device's address, this configuration file uses
the proxy agent's default values.

For further information on configuration defaults, 
and on setting up the proxy agent's device address, see:
`http://volttron.readthedocs.io/en/develop/core_services/drivers/BACnet-Proxy-Agent.html?highlight=bacnet%20proxy#device-addressing`

In [ ]:
# Replace this address with the IP and (optional) port number of VOLTTRON's BACnet virtual device.
# This is *not* the IP address of any physical device.
# A subnet mask may be used if needed, e.g. 192.168.1.2/24 matches any device on the 192.168.1 subnet.
# The default port number is 47808.
bacnet_proxy_ip_address = '192.168.14.65/24'

config = """{{
    "agentid": "bacnet_proxy",
    # (Required) Use this network interface for the virtual device.
    "device_address": "{0}" 
}}""".format(bacnet_proxy_ip_address)
print "config = {}".format(config)

bacnet_proxy_config_path = vhome + '/my_bacnet_proxy.config'
with open(bacnet_proxy_config_path, 'w') as file:
    file.write(config)
print('BACnet proxy configuration written to {}\n'.format(bacnet_proxy_config_path))

# Configure a BACnet device driver

Create and install a configuration for the BACnet device.

In [ ]:
# Change this value to the IP address of the BACnet device
bacnet_device_ip = '192.168.14.2'

def install_driver_csv(name=None, csv=None):
    _sh('volttron-ctl config store platform.driver {0} {1} --csv'.format(name, csv))

def install_driver_config(name=None, config=None):
    _sh('volttron-ctl config store platform.driver {0} {1}'.format(name, config))

# Create a CSV points file for the device
points = '''Point Name,Volttron Point Name,Units,Unit Details,BACnet Object Type,Property,Writable,Index,Notes
DA1-P,DischargeAirStaticPressure,inchesOfWater,-0.20 to 5.00,analogInput,presentValue,FALSE,3000108,Resolution: 0.001
DA-T,DischargeAirTemperature,degreesFahrenheit,-50.00 to 250.00,analogInput,presentValue,FALSE,3000109,Resolution: 0.1
MA-T,MixedAirTemperature,degreesFahrenheit,-50.00 to 250.00,analogInput,presentValue,FALSE,3000116,Resolution: 0.1
OA-H,OutdoorAirHumidity,percentRelativeHumidity,0.00 to 100.00,analogInput,presentValue,FALSE,3000117,Resolution: 0.1
PH-T,PreheatTemperature,degreesFahrenheit,-50.00 to 250.00,analogInput,presentValue,FALSE,3000119,Resolution: 0.1
RA-T,ReturnAirTemperature,degreesFahrenheit,-50.00 to 250.00,analogInput,presentValue,FALSE,3000120,Resolution: 0.1
RA-H,ReturnAirHumidity,percentRelativeHumidity,0.00 to 100.00,analogInput,presentValue,FALSE,3000124,Resolution: 0.1
CLG-O,CoolingValveOutputCommand,percent,0.00 to 100.00 (default 0.0),analogOutput,presentValue,TRUE,3000107,Resolution: 0.1
MAD-O,MixedAirDamperOutputCommand,percent,0.00 to 100.00 (default 0.0),analogOutput,presentValue,TRUE,3000110,Resolution: 0.1
PH-O,PreheatValveOutputCommand,percent,0.00 to 100.00 (default 0.0),analogOutput,presentValue,TRUE,3000111,Resolution: 0.1
RH-O,ReheatValveOutputCommand,percent,0.00 to 100.00 (default 0.0),analogOutput,presentValue,TRUE,3000112,Resolution: 0.1
SF-O,SupplyFanSpeedOutputCommand,percent,0.00 to 100.00 (default 0.0),analogOutput,presentValue,TRUE,3000113,Resolution: 0.1
RF-O,ReturnFanSpeedOutputCommand,percent,0.00 to 100.00 (default 0.0),analogOutput,presentValue,TRUE,3000122,Resolution: 0.1
OAT,OutdoorAirTemperature,days,No limits. (default 70.0),analogValue,presentValue,FALSE,3000019,
Test error data point,UnknownEnergyValue,noUnits,No limits. (default 0.0),analogValue,presentValue,FALSE,3000025,
AV1,AnalogVariable1,noUnits,No limits. (default 0.0),analogValue,presentValue,FALSE,3000026,
Programming.AV1,ProgrammingAnalogVariable1,noUnits,No limits. (default 0.0),analogValue,presentValue,FALSE,3000028,
AV2,AnalogVariable2,noUnits,No limits. (default 0.0),analogValue,presentValue,FALSE,3000033,
AHU-STATE,AHUState,State,State count: 10 (default 1),multiStateValue,presentValue,FALSE,3000186,"1=Satisfied, 2=Econ, 3=Econ+Mech, 4=HX Cool+Mech, 5=HX Heat, 6=HX Heat+Preheat, 7=HX Heat+Preheat+Reheat, 8=Cooling Idle, 9=Heating Idle, 10=Temperature Unreliable"'''
# print('points file contents = {}'.format(points))

csv_path = vhome + '/my_bacnet.csv'
with open(csv_path, 'w') as file:
    file.write(points)
print('BACnet points file written to {}\n'.format(csv_path))

# Create a config file for the device
config = """{{
    "driver_config": {{
        "device_address": "{0}",
        "device_id": 500,
        "timeout": 10
    }},
    "driver_type": "bacnet",
    "registry_config":"config://{1}",
    "interval": 60,
    "timezone": "UTC"
}}""".format(bacnet_device_ip, 'my_bacnet.csv')
print("config = {}".format(config))

config_path = vhome + '/my_bacnet.config'
with open(config_path, 'w') as file:
    file.write(config)
print('BACnet configuration written to {}'.format(config_path))

# Store the configurations in the master driver.
print('\nWait for the platform driver config to display, then confirm that this config appears in it...')
install_driver_csv(name='my_bacnet.csv', csv=csv_path)
install_driver_config(name='devices/my_bacnet', config=config_path)

# List the driver configuration to confirm that the drivers were installed successfully.
_print_sh('volttron-ctl config list platform.driver')

# Setup: Install Agents

Install each agent employed by the Collector: a MasterDriver, a ForwardHistorian, a BACnetProxy, and 2 Volttron Central agents.

In [ ]:
print('Wait for the list to be displayed, then confirm that all of these agents appear in it...')

def install_agent(dir=None, id=None, config=None, tag=None):
    script_install_command = 'python scripts/install-agent.py -s {0} -i {1} -c {2} -t {3} -f'
    _sh(script_install_command.format(dir, id, config, tag))
    print('Installed {}'.format(tag))

# Install the MasterDriver agent which runs the Bacnet driver
install_agent(dir=vroot+'/services/core/MasterDriverAgent/',
              id='platform.driver',
              config=vroot+'/services/core/MasterDriverAgent/master-driver.agent',
              tag='platform.driver')

# Install a ForwardHistorian agent that forwards metrics to another VOLTTRON instance
install_agent(dir=vroot+'/services/core/ForwardHistorian',
              id='forward_historian',
              config=vhome+'/my_bacnet_forwarder.config',
              tag='forward_historian')

# Install a BACnet proxy agent that communicates with BACnet devices
install_agent(dir=vroot+'/services/core/BACnetProxy',
              id='platform.bacnet_proxy',
              config=vhome+'/my_bacnet_proxy.config',
              tag='platform.bacnet_proxy')

# Install a Platform Agent
install_agent(dir=vroot+'/services/core/VolttronCentralPlatform',
              id='platform.agent',
              config=vroot+'/services/core/VolttronCentralPlatform/config', 
              tag='vcp')

# Install a Volttron Central Agent
install_agent(dir=vroot+'/services/core/VolttronCentral',
              id='volttron.central',
              config=vroot+'/services/core/VolttronCentral/config', 
              tag='vc')

# List agent status to verify that the agents were installed successfully.
_print_sh('volttron-ctl status', stderr=subprocess.STDOUT)

# Setup: Get the Collector's forward_historian Credentials

The Collector's ForwardHistorian agent needs to authenticate to the Aggregator. Authentication is facilitated by adding the agent's credentials to the Aggregator's auth.json file.

Copy the PUBLICKEY from the command output below. On the Aggregator, run `volttron-ctl auth add` from the command line. When prompted for credentials, paste the key.

In [ ]:
_print_sh('volttron-ctl auth publickey --tag forward_historian')

# Execution: Refresh Variables and Stop Agents

Before starting up the agents, refresh all variables and make sure that all agents are stopped.

In [ ]:
print('Make a fresh start - refresh variable definitions, shut down any running agents, refresh the database')

import datetime
import json
import os
import pprint
import sqlite3
import subprocess
import sys
import time

# Define a "run this shell command" method, wrapping subprocess.check_output()
def _sh(shell_command, shell=True, stderr=None):
    try:
        return_value = subprocess.check_output(shell_command, shell=shell, stderr=stderr)
    except Exception, err:
        print('Shell command failed: {}', shell_command)
        print(err)
        return_value = 'Error'
    return return_value

# Same as _sh(), except that this also prints the command output, preceded by an optional label.
def _print_sh(shell_command, label=None, **kwargs):
    print('{0}: {1}\n'.format(label+':' if label else '', _sh(shell_command, **kwargs)))

# Set up local variables vhome and vroot.
# The environment variables VOLTTRON_ROOT and VOLTTRON_HOME should already be defined -- see above.
vroot = %env VOLTTRON_ROOT
vhome = %env VOLTTRON_HOME
print("VOLTTRON_ROOT={}".format(vroot))
print("VOLTTRON_HOME={}".format(vhome))

# Define a VIP_SOCKET environment variable for use while installing and running agents.
socket_name = 'ipc://' + vhome + '/run/vip.socket'
%env VIP_SOCKET=$socket_name

# Run from the VOLTTRON root directory.
os.chdir(vroot)

# Shut down all agents.
_sh('volttron-ctl shutdown')

# List agent status to verify that the status of each agent is 0 or blank.
_print_sh('volttron-ctl status', stderr=subprocess.STDOUT)

# Execution: Start the agents

In [ ]:
print('Wait for the list to be displayed, then confirm that each started agent is running...')

_sh('volttron-ctl start --tag platform.driver')
_sh('volttron-ctl start --tag platform.bacnet_proxy')
_sh('volttron-ctl start --tag forward_historian')
_sh('volttron-ctl start --tag vcp')
_sh('volttron-ctl start --tag vc')

# List agent status to verify that the started agents have status "running".
_print_sh('volttron-ctl status', stderr=subprocess.STDOUT)

# Shutdown: Stop all agents

In [ ]:
# Stop all agents.
_sh('volttron-ctl shutdown')

# Verify that all agents have been stopped.
_print_sh('volttron-ctl status', stderr=subprocess.STDOUT)